In [4]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

import os
import pandas as pd

from itertools import product

import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

def print_answer(status, correct, answ, V, K, M):
    print(status)
    if correct == "correct":
        for i, j, k, t in product(V, V, K, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)
    
def set_time(t0, T, Contracts):
    t = {}
    points = set()
    points.add(t0)
    points.add(T)
    for c in Contracts:
        points.add(c['t_s'])
        points.add(c['t_s'] + c['t'])
    points = sorted(list(points))
    for i in range(len(points) - 1):
        e = points[i + 1] - points[i]
        C = set()
        for c in range(len(Contracts)):
            if points[i] >= Contracts[c]['t_s'] and points[i] < Contracts[c]['t_s'] + Contracts[c]['t']:
                C.add(c)
        t[i] = {'em': e, 'Ctm': C}
    return points, t

def unit_measure(strr):
    mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

random.seed(3)

H1=nx.read_graphml('change_topo/cBren.graphml')
H = nx.Graph(H1)

print("Info about topology:")
print("Name   " + "cBren")
print("Number of nodes" + str(H.nodes()))
print("Number of edges" + str(H.edges()))

Bandwidth = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))
BackFlows = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

for edge in H.edges(data = True):
    data_edge = H.get_edge_data(edge[0], edge[1])
    bndw = unit_measure(data_edge['LinkLabel'])
    bflw = data_edge['BackgroundFlow']
#     print(Bandwidth.shape)
#     print(edge)
#     print(Bandwidth[0][237])
    Bandwidth[int(edge[1])][int(edge[0])] = bndw
    Bandwidth[int(edge[0])][int(edge[1])] = bndw
    BackFlows[int(edge[1])][int(edge[0])] = bflw
    BackFlows[int(edge[0])][int(edge[1])] = bflw
    
    
Contracts = [{'volume': 60, 't_s':  60, 't':  60, 'A': 3, 'B': 8},
             {'volume': 100, 't_s':  20, 't':  50, 'A': 7, 'B': 21},
             {'volume': 50, 't_s':  10, 't':  50, 'A': 5, 'B': 26},
             {'volume': 100, 't_s':  0, 't':  50, 'A': 4, 'B': 3},
             {'volume': 100, 't_s':  100, 't':  50, 'A': 9, 'B': 17},
             {'volume': 100, 't_s':  0, 't':  20, 'A': 12, 'B': 19},
             {'volume': 100, 't_s':  50, 't':  50, 'A': 8, 'B': 5},
             {'volume': 100, 't_s':  70, 't':  50, 'A': 26, 'B': 8},
             {'volume': 100, 't_s':  40, 't':  50, 'A': 6, 'B': 30},
             {'volume': 100, 't_s':  30, 't':  50, 'A': 33, 'B': 12}]

mmin = 10000000
mmax = 0
for c in Contracts:
    if c['t_s'] < mmin:
        mmin = c['t_s']
    if c['t_s'] + c['t'] > mmax:
        mmax = c['t_s'] + c['t']
        
V = set(range(Bandwidth.shape[0])) # число вершин
K = set(range(len(Contracts)))     # количество контрактов К
# t0 = 0                             # начало отсчета t0
# T = 1000                           # конец отсчета в сек
t0 = mmin
T = mmax
timeline, Intervals = set_time(t0, T, Contracts)
M = set(range(len(Intervals)))
print("Timeline ")
print(timeline)
print("Time intervals ")
print(Intervals)

print("Pyomo evaluationg ...")
status, correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(status, correct_pyomo, answ_pyomo, V, K, M)

print("MIP evaluationg ...")
status, correct_mip, answ_mip, time_mip = mip.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(status, correct_mip, answ_mip, V, K, M)

print("Does MIP and Pyomo answers are equal?")
print((answ_mip == answ_pyomo).all())

Info about topology:
Name   cBren
Number of nodes['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']
Number of edges[('0', '3'), ('0', '28'), ('1', '3'), ('1', '4'), ('1', '6'), ('1', '7'), ('2', '34'), ('2', '3'), ('3', '8'), ('3', '14'), ('3', '21'), ('3', '22'), ('3', '29'), ('3', '30'), ('5', '14'), ('9', '15'), ('10', '16'), ('10', '17'), ('10', '11'), ('11', '12'), ('11', '13'), ('11', '21'), ('15', '22'), ('17', '25'), ('18', '23'), ('19', '21'), ('20', '21'), ('22', '25'), ('23', '25'), ('24', '25'), ('25', '26'), ('27', '35'), ('27', '28'), ('31', '34'), ('32', '34'), ('33', '34'), ('34', '35'), ('35', '36')]
Timeline 
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150]
Time intervals 
{0: {'em': 10, 'Ctm': {3, 5}}, 1: {'em': 10, 'Ctm': {2, 3, 5}}, 2: {'em': 10, 'Ctm': {1, 2, 3}}, 3: {'em': 10, 'Ctm': {1, 2, 3, 9}},

answer[33][34][9][3] = 2.0
answer[33][34][9][4] = 2.0
answer[33][34][9][5] = 2.0
answer[33][34][9][6] = 2.0
answer[33][34][9][7] = 2.0
answer[34][2][9][3] = 2.0
answer[34][2][9][4] = 2.0
answer[34][2][9][5] = 2.0
answer[34][2][9][6] = 2.0
answer[34][2][9][7] = 2.0
Does MIP and Pyomo answer are equal?
True
